# 인스타그램 제주맛집

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [2]:
import unicodedata, re
from tqdm.notebook import tqdm

In [3]:
insta_url = 'https://www.instagram.com'
driver = webdriver.Chrome('chromedriver')
driver.get(insta_url)
time.sleep(1)

### 1. 로그인

In [4]:
with open('password.txt') as f:
    password = f.read()

In [5]:
email = 'ihessiek@gmail.com'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [6]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [7]:
# 로그인 정보 저장 - 나중에 하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [8]:
# 알림 설정 - 나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 2. 제주맛집

In [9]:
keyword = '제주맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%A7%9B%EC%A7%91'

In [10]:
driver.get(url)
time.sleep(5)

### 3. 첫번째 게시글

In [11]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [12]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [13]:
# 본문 내용 가져오기
try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'광고제주도에서 즐기는 랍스터만찬 랍스터빈 !!🔸주소 : 제주시 애월읍 애월해안로 752🔸영업 : 오전11시~오후10시🔸전화: 064-743-1979🔸연중무휴 @lobstervin#애월맛집 #애월카페 #애월읍맛집 #제주애월맛집 #제주도맛집 #제주맛집#제주카페 #외도맛집 #하귀맛집#한림맛집 #한림카페 #애월해안도로맛집#애월가볼만한곳 #제주공항맛집 #제주데이트 #제주공항근처맛집 #한담맛집 #제주해안도로#제주맛집추천 #제주도맛집추천 #제주도민맛집'

- emoji 는 4-byte로 됨

In [14]:
# 2) 본문 내용에서 해쉬태그 가져오기, '#[^\s#,\\]+'
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#애월맛집',
 '#애월카페',
 '#애월읍맛집',
 '#제주애월맛집',
 '#제주도맛집',
 '#제주맛집',
 '#제주카페',
 '#외도맛집',
 '#하귀맛집',
 '#한림맛집',
 '#한림카페',
 '#애월해안도로맛집',
 '#애월가볼만한곳',
 '#제주공항맛집',
 '#제주데이트',
 '#제주공항근처맛집',
 '#한담맛집',
 '#제주해안도로',
 '#제주맛집추천',
 '#제주도맛집추천',
 '#제주도민맛집']

In [15]:
# 3) 작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-12-06'

In [16]:
# 4) 좋아요 수 가져오기: 좋아요 1,296개
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

0

In [17]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

''

In [18]:
row = [content, date, like, place, tags]
row

['광고제주도에서 즐기는 랍스터만찬 랍스터빈 !!🔸주소 : 제주시 애월읍 애월해안로 752🔸영업 : 오전11시~오후10시🔸전화: 064-743-1979🔸연중무휴 @lobstervin#애월맛집 #애월카페 #애월읍맛집 #제주애월맛집 #제주도맛집 #제주맛집#제주카페 #외도맛집 #하귀맛집#한림맛집 #한림카페 #애월해안도로맛집#애월가볼만한곳 #제주공항맛집 #제주데이트 #제주공항근처맛집 #한담맛집 #제주해안도로#제주맛집추천 #제주도맛집추천 #제주도민맛집',
 '2021-12-06',
 0,
 '',
 ['#애월맛집',
  '#애월카페',
  '#애월읍맛집',
  '#제주애월맛집',
  '#제주도맛집',
  '#제주맛집',
  '#제주카페',
  '#외도맛집',
  '#하귀맛집',
  '#한림맛집',
  '#한림카페',
  '#애월해안도로맛집',
  '#애월가볼만한곳',
  '#제주공항맛집',
  '#제주데이트',
  '#제주공항근처맛집',
  '#한담맛집',
  '#제주해안도로',
  '#제주맛집추천',
  '#제주도맛집추천',
  '#제주도민맛집']]

### 4. 다음 게시글로 이동

In [19]:
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button').click()
time.sleep(2)

In [20]:
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button').click()
time.sleep(2)

In [21]:
# 5을 하기 위해 popup 화면을 없앰(즉, X 를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

### 5. 반복문으로 데이터 가져오기

In [22]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, date, like, place, tags]

In [23]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [24]:
results = []
for _ in tqdm(range(500)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button/div/span').click()
        time.sleep(3)
    except:
        time.sleep(3)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button/div/span').click()
        time.sleep(3)

  0%|          | 0/500 [00:00<?, ?it/s]

In [26]:
driver.close()

InvalidSessionIdException: Message: invalid session id


### 6. 데이터프레임으로 만들고 중복 제거

In [27]:
df = pd.DataFrame(results, columns=['content', 'date', 'like', 'place', 'tags'])
df.head()

,content,date,like,place,tags
0,광고제주도에서 즐기는 랍스터만찬 랍스터빈 !!🔸주소 : 제주시 애월읍 애월해안로 7...,2021-12-06,0,,"[#애월맛집, #애월카페, #애월읍맛집, #제주애월맛집, #제주도맛집, #제주맛집,..."
1,영업시간 네이버 참고 #제주맛집 #제주도맛집#에코랜드 #사려니숲길 #코코몽...,2021-12-07,140,,"[#제주맛집, #제주도맛집, #에코랜드, #사려니숲길, #코코몽에코파크제주, #스누..."
2,"제주 갈치요리하면 요기 !!!비주얼 최고맛집이라불리는 갈치요리맛집""성산을본갈치""성산...",2021-12-07,0,,[#성산을본갈치주소]
3,#제주맛집🍜제주도민선정맛집 ! 제주순메밀막국수🍜🎄직접 짜서 만드는 들기름으로 건강 ...,2021-12-08,0,제주순메밀막국수,"[#제주맛집🍜제주도민선정맛집, #카멜리아힐맛집.밀가루, #안덕면맛집, #제주도서쪽맛..."
4,#리그램 - @gunwoo_o942021. 11. 30 제주 1일❤️..또 가고싶다...,2021-12-07,328,월정리해변,"[#리그램, #안녕하샌]"


In [31]:
df.drop_duplicates(subset='content', keep='first', inplace=True) # drop_duplicates 특정 열에서 중복되는 요소 제거
df.shape

(322, 5)

In [29]:
df.to_csv('인스타-제주맛집.csv', index=False)